In [1]:
import numpy as np
import pandas as pd
from astropy.io import ascii
from scipy import stats
from scipy.optimize import minimize,curve_fit
from scipy.stats import gaussian_kde as kde
np.random.seed(9620)

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
sns.set(style='ticks', context='talk')
plt.style.use("../paper.mplstyle")

from multiprocessing import Pool
from tqdm import tqdm

from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord, Galactic
from astropy import units as u

from warnings import filterwarnings
filterwarnings(action='ignore')

In [19]:
data2 = pd.read_csv('../data/A2/A2_ver4_CosmoFit.csv').set_index('ID')
data2_noinfo = pd.read_csv('data/A2_missingInfo.csv')
data2_noinfo2 = pd.read_csv('data/A2_missingInfo2.csv')

In [24]:
data2

,Unnamed: 0,zCMB,mu,e_mu,RA,DEC,Sample,GID,Host,morphology,...,Notes,Host_RA,Host_DEC,cutsmask,mu_cosmo,HR,HR_err,H0,missinginfo,noinfo
ID,,,,,,,,,,,,,,,,,,,,,
sn2002bf,1,0.024883,34.502538,0.137125,153.927770,55.638313,Old,0,CGCG 266-031,Sb,...,NaN,153.926060,55.667485,True,34.393344,0.109195,0.158685,100.527254,NaN,NaN
sn2001v,2,0.016187,33.098855,0.101755,179.317053,25.201505,Old,0,NGC 3987,Sb,...,NaN,NaN,NaN,True,33.445389,-0.346534,0.129351,100.527254,NaN,NaN
sn2001cz,3,0.012640,33.311057,0.149839,191.868333,-39.565327,Old,1134,NGC 4679,Sbc,...,NaN,191.876042,-39.570861,True,32.902494,0.408563,0.169791,100.527254,NaN,NaN
sn2001cn,4,0.014683,33.177658,0.154780,281.664080,-65.808204,Old,0,IC 4758,Sc,...,NaN,281.576125,-65.756667,True,33.231252,-0.053594,0.174168,100.527254,NaN,NaN
sn2001bt,5,0.013740,33.077243,0.157338,288.502016,-59.317341,Old,0,IC 4830,Sb,...,NaN,288.452333,-59.294361,True,33.085508,-0.008265,0.176444,100.527254,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017ln,409,0.027420,34.554388,0.078062,174.640750,25.396350,Foundation,0,2MASX J11383367+2523532,Sb,...,NaN,174.640490,25.398098,True,34.608246,-0.053858,0.111674,100.527254,NaN,NaN
2017lv,410,0.029890,34.839497,0.096921,176.110580,-28.457560,Foundation,0,ESO 440- G 001,Sab,...,NaN,176.107875,-28.462111,True,34.799523,0.039974,0.125583,100.527254,NaN,NaN
2017me,411,0.033930,35.296564,0.078062,171.977170,27.345770,Foundation,0,KUG 1125+276,Sb,...,NaN,171.980186,27.347021,True,35.081300,0.215264,0.111674,100.527254,NaN,NaN


In [31]:
data2['noinfo']=np.nan
for name in data2.index.values:
    if name[:2] == 'sn' or name[:2]=='SN':
        name = name[2:]
    else:
        name = name
        if name[0]!='2' and name[0]!='1':
            print(name)
    if name in data2_noinfo['SNe'].values or name in data2_noinfo2['SNe']:
        data2.loc[name,'noinfo'] = True
data_final = data2[data2['noinfo']!=True]

In [32]:
data_final = data_final.drop(columns=['Unnamed: 0','missinginfo','noinfo','GID'])

In [34]:
data_final.to_csv('data/A2_rev1.csv',index=True)